# Regresión en PyTorch

En este cuaderno, usamos la librería PyTorch **diferenciación automática** para ajustar una recta a puntos de datos. Así, aquí usamos el cálculo para resolver el mismo problema de regresión que usamos para resolver la Pseudoinversa de Moore-Penrose en el [*Cuaderno de Álgebra Lineal II*](https://github.com/joanby/matematicas-ml/blob/master/notebooks/2-linear-algebra-ii.ipynb).

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
x=torch.tensor([0,1,2,3,4,5,6,7.])
x

Los valores $y$ se crearon utilizando la ecuación de una recta $y = mx + b$. De este modo, sabemos cuáles son los parámetros del modelo que hay que aprender, digamos, $m = -0,5$ y $b = 2$. Se ha añadido ruido aleatorio distribuido normalmente para simular el error de muestreo:

In [ ]:
#y=-0.5*x+2 + torch.normal(mean=torch.zeros(8),std=0.2)

Para la reproducibilidad de esta demostración, he aquí un ejemplo fijo de los valores $y$ obtenidos ejecutando la línea comentada anteriormente:

In [ ]:
y=torch.tensor([1.86,1.31,.62,.33,.09,-.67,-1.23,-1.37]) #puntuacion de olvido delpaciente

In [ ]:
y

In [ ]:
fig,ax=plt.subplots()
plt.title('Ensayo Clinico')
plt.xlabel('Dosis de Droga')
plt.ylabel('Nivel de Olvido')
ax.scatter(x,y)

Inicializar el parámetro de pendiente $m$ con un valor «aleatorio» de 0,9...

**N.B.**: En esta sencilla demostración, podríamos empezar adivinando valores de parámetros aproximadamente correctos. O podríamos utilizar un método algebraico (por ejemplo, la pseudoinversa de Moore-Penrose) o estadístico (por ejemplo, la regresión por mínimos cuadrados ordinarios) para resolver los parámetros rápidamente. Sin embargo, esta pequeña demostración de aprendizaje automático con dos parámetros y ocho puntos de datos puede ampliarse a millones de parámetros y millones de puntos de datos. Los demás enfoques -adivinanzas, álgebra, estadística- no se acercan a este escalado).

In [ ]:
m=torch.tensor([0.9]).requires_grad_()
m

...y hacer lo mismo para el parámetro $y$-intercepto $b$:

In [ ]:
b=torch.tensor([0.1]).requires_grad_()
b

In [ ]:
def regression(my_x,my_m,my_b):
  return my_m*my_x+my_b

In [ ]:
def regression_plot(my_x,my_y,my_m,my_b):
  fig,ax=plt.subplots()
  plt.scatter(my_x,my_y)
  x_min,x_max=ax.get_xlim()
  y_min=regression(x_min,my_m,my_b).detach().item()
  y_max=regression(x_max,my_m,my_b).detach().item()

  ax.set_xlim([x_min,x_max])
  plt.plot([x_min,x_max],[y_min,y_max])


In [ ]:
regression_plot(x,y,m,b)

### Machine Learning
En cuatro pasos muy sencillos :)


**Paso 1**: Paso hacia adelante

In [ ]:
yhat=regression(x,m,y) #es el modelo predictivo
yhat # es la estimacion de lo que y debe ser usando m=0.9 y b=0.1

**Paso 2**: Comparar $\hat{y}$ con $y$ verdadero para calcular el coste $C$

Existe un método PyTorch `MSELoss`, pero vamos a definirlo nosotros mismos para ver cómo funciona. El coste MSE se define por: $$C = \frac{1}{n} \sum_{i=1}^n (\hat{y_i}-y_i)^2 $$

In [ ]:
def mse(my_yhat,my_y): #mse mean square error (error cuadrado medio) perdida analisamos cuanto se equivoca el modelo
  sigma=torch.sum((my_yhat-my_y)**2) #y puntos de reales de la funcion y my_yhat -> los valores a estimar
  return sigma/len(my_y)

In [ ]:
C=mse(yhat,y)
C

**Paso 3**: Utilizar autodiff para calcular el gradiente de $C$ en función de los parámetros.

In [ ]:
C.backward() #diferenciamos hacia atras la funcion de costes

In [ ]:
m.grad #ahora pedimos a gradiente de la funcion de costes
#reducir la m es fundamental para reducir la funcion de costes

In [ ]:
b.grad
b

**Paso 3**: Utilizar autodiff para calcular el gradiente de $C$ en función de los parámetros.

In [ ]:
optimizer=torch.optim.SGD([m,b],lr=0.01) #gradiente descendente estocastico
#optim es un metodo muy popular para reducir la gradiente
#lr=learning rate

In [ ]:
optimizer.step() #damos el siguiente paso para corregir m y b usando las gradientes calculados previamente

Confirme que los parámetros se han ajustado con sensatez:

In [ ]:
m

In [ ]:
b

In [ ]:
regression_plot(x,y,m,b)

Podemos repetir los pasos 1 y 2 para confirmar que el coste ha disminuido:

In [ ]:
C=mse(regression(x,m,b),y)
C

In [ ]:
epochs=1000 #ronda de entrenamiendo
for epoch in range(epochs):
  optimizer.zero_grad()#Reinicial gradiente a zero; si no, se acumularia
  yhat=regression(x,m,b) # Paso1
  C=mse(yhat,y) # Paso 2

  C.backward() #Paso 3 #hallamos la pendiente de funcion de costes la m y la b
  optimizer.step() #PAso 4

  print('Epoch {}, coste {}, m grad {}, b grad {}'.format(epoch, '%.3g' % C.item(), '%.3g' % m.grad.item(), '%.3g' % b.grad.item()))


In [ ]:
regression_plot(x,y,m,b)

In [ ]:
m.item() #obtener los valores finales o flotantes

In [ ]:
b.item()

**N.B.**: El modelo no se aproxima perfectamente a la pendiente (-0,5) y a la intersección $y$ (2,0) utilizadas para simular los resultados $y$ en la parte superior de este cuaderno. Esto refleja la imperfección de la muestra de ocho puntos de datos debido a la adición de ruido aleatorio durante el paso de simulación. En el mundo real, la mejor solución sería muestrear puntos de datos adicionales: Cuantos más datos muestreemos, más precisas serán nuestras estimaciones de los verdaderos parámetros subyacentes.

Ejercicios:
1.- Use Pytorch (o TensorFlow, si lo desea) para encontrar la pendiente de y=x´2+2x+2 donde x=2

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
x=torch.linspace(-10.,10.,1000)

In [ ]:
def f(my_x):
  my_y=my_x**2+2*x+2

In [ ]:
y=f(x)

In [ ]:
y=x**2+2*x+2

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.plot(x,y)
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')
ax.set_title('y=x**2+2*x+2')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.grid(color='#dadada',linestyle='--')

In [ ]:
f(torch.tensor([2]))

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.plot(x,y)
ax.scatter(2,10,color='r')
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')
ax.set_title('y=x**2+2*x+2')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.grid(color='#dadada',linestyle='--')



In [ ]:
f(5)

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.plot(x,y)
ax.scatter(2,10,color='y')
ax.scatter(5,37,color='r')
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')
ax.set_title('y=x**2+2*x+2')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.grid(color='#dadada',linestyle='--')


In [ ]:
m=(37-10)/(5-2)
m

In [ ]:
b=y-m*x

In [ ]:
b=37-9*5
b

In [ ]:
my_y=m*x+b

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.scatter(2,10,color='y')
plt.scatter(5,37,color='r')
plt.plot(x,y)
plt.plot(x,my_y)
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')

In [ ]:
f(2.1)

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.plot(x,y)
ax.scatter(2,10,color='y')
ax.scatter(2.1,10.61 ,color='r')
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')
plt.grid(color='#dadada',linestyle='--')

In [ ]:
m=(10.61-10)/(2.1-2)
m

In [ ]:
b=10.61-m*2.1
b

In [ ]:
line_y=m*x+b

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.scatter(2,10,color='y')
plt.scatter(2.1,10.61 ,color='r')
plt.plot(x,y)
plt.plot(x,line_y)
plt.axvline(x=0,color='lightgray')
plt.axhline(y=0,color='lightgray')
plt.grid(color='#dadada',linestyle='--')

In [ ]:
delta_x=0.00001
delta_x


In [ ]:
#delta_x=x_2-x_1


In [ ]:
x_2=delta_x+2
x_2

In [ ]:
y_1=f(2)

In [ ]:
y_2=f(2.00001)

In [ ]:
m=(8.00001-9)/(2.00001-2)
m

In [ ]:
b=8.000001-m*(2.000001)
b

In [ ]:
line_y=m*x+b

In [ ]:
fig,ax=plt.subplots()
plt.xlim([-10,10])
plt.ylim([-10,140])
plt.scatter(2,10,color='y')
plt.scatter(2.000001,10 ,color='r')
plt.plot(x,y)
plt.plot(x,line_y)

Para encontrar la pendiente (derivada) de la función y=x2+2x+2 en el punto x=2 usando PyTorch, seguiremos estos pasos:

In [ ]:
import torch

# 1. Definir la variable x con requires_grad=True (para calcular gradientes)

In [ ]:
x=torch.tensor(2.0,requires_grad=True)
x

# 2. Definir la función y

In [ ]:
y=x**2+2*x+2

# 3. Calcular la derivada (pendiente)

In [ ]:
y.backward()


# 4. Obtener la pendiente en x=2

In [ ]:
pendiente=x.grad.item()

In [ ]:
f'La pendiente en x=2 es {pendiente}'